<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Лемматизация-и-регулляризация-признаков" data-toc-modified-id="Лемматизация-и-регулляризация-признаков-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Лемматизация и регулляризация признаков</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#Random-Forest-Classifier" data-toc-modified-id="Random-Forest-Classifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Random Forest Classifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np


import torch
import transformers

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem.snowball import EnglishStemmer
from pymystem3 import Mystem


from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


## Лемматизация и регулляризация признаков

Разделим данные на обучающую и тестовую выборки.

In [6]:
X = df['text'].copy()
y = df['toxic']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=.2,
    random_state=12345
)

In [8]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text) 
    row = " ".join(text.split())
    return row

In [9]:
X_train = X_train.values.astype('U')
X_test = X_test.values.astype('U')

In [10]:
for i in range(len(X_train)):
    X_train[i] = clear_text(X_train[i])

In [11]:
for i in range(len(X_test)):
    X_test[i] = clear_text(X_test[i])

In [12]:
stemmer = EnglishStemmer(ignore_stopwords=False)

In [13]:
for i in range(len(X_train)):
    word_list = nltk.word_tokenize(X_train[i])
    X_train[i] = ' '.join([stemmer.stem(w) for w in word_list])

In [14]:
for i in range(len(X_test)):
    word_list = nltk.word_tokenize(X_test[i])
    X_test[i] = ' '.join([stemmer.stem(w) for w in word_list])

Загрузим базу стоп слов

In [15]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Векторизуем данные с учетом стоп-слов. Применим метод fit к обычающей выборке, а затем метод transform к обучающему и тестовому набору данных.

In [16]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(X_train)

In [17]:
X_train = tf_idf.transform(X_train)

X_test = tf_idf.transform(X_test)

## Обучение

### LogisticRegression

In [18]:
model_line = LogisticRegression(
    random_state=12345,
    solver = 'liblinear',
    class_weight='balanced'
)
model_line.fit(X_train, y_train)

predict_line = model_line.predict(X_test)


In [19]:
print('F1-score LogisticRegression:',(f1_score(pd.Series(predict_line), y_test)))

F1-score LogisticRegression: 0.7538337368845842


### Random Forest Classifier

In [20]:
# parameters = {
#     'max_depth': [20, 25],
#     'n_estimators': [150, 200]
# }

# model_forest = GridSearchCV(
#     RandomForestClassifier(random_state=12345),
#     parameters,
#     cv=5,
#     scoring='f1',
#     n_jobs=-1,
# )

# model_forest.fit(X_train, y_train)

# best_params = model_forest.best_params_

# print("Лучшие параметры: ", model_forest.best_params_)
# print("Лучшее занчение F1-score: ", model_forest.best_score_)

In [21]:
model_forest = RandomForestClassifier(max_depth = 25, n_estimators=150,  class_weight='balanced', random_state=12345)

model_forest.fit(X_train, y_train)

predict_forest = model_forest.predict(X_test)

In [22]:
print('F1-score RandomForestClassifier:',(f1_score(pd.Series(predict_forest), y_test)))

F1-score RandomForestClassifier: 0.4316790203029327


## Выводы

- Данные успешно загружены
- Проведена лемматизация и регулляризация текстов
- Удалены стоп слова  
- Корпус векторизован с помощью TfidfVectorizer
- Построены модели LogisticRegression и RandomForestClassifier
- F1-score достиг лучшего значения у LogisticRegression: 0.754

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны